In [ ]:
!pip install datasets faiss-cpu==1.7.4 chromadb==0.4.22 sentence-transformers==2.3.1

In [ ]:
from datasets import load_dataset

qna_dataset = load_dataset("sadeem-ai/arabic-qna")

news_dataset = load_dataset("arbml/SANAD")

In [ ]:
news_dataset = news_dataset.filter(lambda example: len(example["Text"]) >= 100)
news_dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'label'],
        num_rows: 288268
    })
})

In [ ]:
news_dataset = news_dataset.shuffle(seed=42)

In [ ]:
qna_dataset = qna_dataset.filter(lambda example: example["has_answer"] == True)

In [ ]:
doc_texts = qna_dataset["train"]["text"] + news_dataset["train"][:30_000]["Text"]
doc_questions = qna_dataset["train"]["question"]

In [ ]:
len(doc_texts)

34037

In [ ]:
metadata = [
    {
        "source": rec["source"],
        "title": rec["title"]
    }
    for rec in qna_dataset["train"]
]

metadata += [
    {
        "source": "",
        "title": "",
    }
    for i in range(30_000)
 ]

In [ ]:
len(metadata)

34037

In [ ]:
docs_ids = [
    str(i)
    for i in range( len(doc_texts) )
]

In [ ]:
len(docs_ids)

34037

In [ ]:
doc_questions[49], docs_ids[49], metadata[49]

('ما عدد العقود في الردهة الكبيرة؟',
 '49',
 {'source': 'https://ar.wikipedia.org/wiki?curid=1291947',
  'title': 'مسجد الجيوشي (القاهرة)'})

## Text to Vectors

In [ ]:
from sentence_transformers import SentenceTransformer

model_id = "sentence-transformers/distiluse-base-multilingual-cased-v2"
dim = 512

# model_id = "asafaya/bert-large-arabic"
# dim = 1024

device = "cuda:0" # "cpu"

model = SentenceTransformer(model_id, device=device)

In [ ]:
encoded_docs = model.encode(doc_texts, show_progress_bar=True)

Batches:   0%|          | 0/1064 [00:00<?, ?it/s]

In [ ]:
encoded_questions = model.encode(doc_questions, show_progress_bar=True)

Batches:   0%|          | 0/127 [00:00<?, ?it/s]

## Vector Databases

### ChromaDB

In [ ]:
import chromadb

chroma_client = chromadb.PersistentClient(path="./chromadb-ar-docs")

In [ ]:
collection = chroma_client.create_collection(
    name="ar_docs_34k",
    metadata={"hnsw:space": "cosine"}
)

In [ ]:
collection.add(
    documents=doc_texts,
    embeddings=encoded_docs,
    metadatas=metadata,
    ids=docs_ids
)

In [ ]:
## Search
question = "ما السبب في صغر الأسنان بالمقارنة مع حجم الفكين؟"
question_embed = model.encode(question)

results = collection.query(
    query_embeddings=question_embed.tolist(),
    n_results=3
)

print(results)

{'ids': [['397', '1534', '30457']], 'distances': [[0.4194859266281128, 0.4194859266281128, 0.5500727891921997]], 'metadatas': [[{'source': 'https://ar.wikipedia.org/wiki?curid=7483665', 'title': 'صغر الأسنان'}, {'source': 'https://ar.wikipedia.org/wiki?curid=7483665', 'title': 'صغر الأسنان'}, {'source': '', 'title': ''}]], 'embeddings': None, 'documents': [['جميع الأسنان ذات حجم طبيعي ولكنها تبدو صغيرة بسبب ضخامة الفكين. قد يكون الصغر النسبي المعمم نتيجة وراثة فك كبير من أحد الوالدين، وأسنان ذات حجم طبيعي من الآخر.', 'جميع الأسنان ذات حجم طبيعي ولكنها تبدو صغيرة بسبب ضخامة الفكين. قد يكون الصغر النسبي المعمم نتيجة وراثة فك كبير من أحد الوالدين، وأسنان ذات حجم طبيعي من الآخر.', 'يعاني البعض من الحرج الشديد بسبب الاصفرار الذي يغطى أسنانهم. فما السبب وما الحل؟\xa0اسباب تغير لون الأسنان:يتغير لون الأسنان ويميل إلى الأصفرار بسبب الصبغات الصفراء الموجودة في بعض المأكولات والمشروبات مثل القهوة والشاي والمياه الغازيه، والتوابل. حيث تمتص طبقة المينا التي تغطي الأسنان هذه الصبغات وتتحول تدريجيا 

### FAISS

In [ ]:
import faiss
import numpy as np
from copy import deepcopy

In [ ]:
norm_encoded_docs = deepcopy(encoded_docs)
faiss.normalize_L2(norm_encoded_docs)

In [ ]:
faiss_index = faiss.IndexIDMap( faiss.IndexFlatIP(dim) )

faiss_index.add_with_ids( norm_encoded_docs, docs_ids )

In [ ]:
question = "ما السبب في صغر الأسنان بالمقارنة مع حجم الفكين؟"
question_embed = model.encode([question])

faiss.normalize_L2(question_embed)

results = faiss_index.search(question_embed, 3)

In [ ]:
print(results)

(array([[0.5805141 , 0.5805141 , 0.44992718]], dtype=float32), array([[ 1534,   397, 30457]]))


In [ ]:
## Save
import pickle

with open("./faiss-ar-docs/index.pickle", "wb") as handle:
    pickle.dump(faiss_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open("./faiss-ar-docs/data.pickle", "wb") as handle:
    pickle.dump({
        "data": doc_texts,
        "docs_ids": docs_ids,
        "metadata": metadata
    }, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
## Load
import pickle

with open("./faiss-ar-docs/index.pickle", "rb") as handle:
    loaded_faiss_index = pickle.load(handle)

with open("./faiss-ar-docs/data.pickle", "rb") as handle:
    loaded_faiss_data = pickle.load(handle)

## Let's Compare

### Retrieving Speed

In [ ]:
import time

**Dim: 512**

---------
`ChromaDB`: 4037 -
12.698187112

`FIASS`: 4037 -
2.043501231999997

---------
**Dim: 1024**

`ChromaDB`: 4037
15.938859471999997

`FIASS`: 4037
4.1828746040000055


In [ ]:
t0 = time.process_time()

for i in range(len(doc_questions)):

    ques = encoded_questions[i]

    results = collection.query(
        query_embeddings=ques.tolist(),
        n_results=3
    )

print("ChromaDB:", len(doc_questions))
print(time.process_time() - t0)

ChromaDB: 4037
13.255644789000002


In [ ]:
t0 = time.process_time()

for i in range(len(doc_questions)):

    ques = encoded_questions[i].reshape(1, dim)

    faiss.normalize_L2(ques)

    results = faiss_index.search(ques, 3)


print("FIASS:", len(doc_questions))
print(time.process_time() - t0)

FIASS: 4037
24.845981306


### Accuracy

`ChromaDB`

```
Model ID: sentence-transformers/distiluse-base-multilingual-cased-v2
----
Valid: 1201
Valid%: 0.2974981421847907
----
Similar: 864
Similar%: 0.21402031211295516
----
InValid: 1972
InValid%: 0.48848154570225416
----


Model ID: asafaya/bert-large-arabic
----
Valid: 586
Valid%: 0.14515729502105523
----
Similar: 427
Similar%: 0.10577161258360168
----
InValid: 3024
InValid%: 0.7490710923953431
----

```

`FAISS`

```
Model ID: sentence-transformers/distiluse-base-multilingual-cased-v2
----
Valid: 1374
Valid%: 0.3403517463462968
----
Similar: 947
Similar%: 0.23458013376269507
----
InValid: 1716
InValid%: 0.4250681198910082
----

Model ID: asafaya/bert-large-arabic
----
Valid: 703
Valid%: 0.17413921228635126
----
Similar: 518
Similar%: 0.12831310378994304
----
InValid: 2816
InValid%: 0.6975476839237057
----
```

In [ ]:
chroma_results = []

for i in range(len(doc_questions)):

    ques = encoded_questions[i]

    results = collection.query(
        query_embeddings=ques.tolist(),
        n_results=3
    )

    chroma_results.append(results)

In [ ]:
chroma_insights = {
    "valid": 0,
    "similar": 0,
    "invalid": 0
}

for i in range(len(doc_questions)):
    true_id = docs_ids[i]
    pred_id = chroma_results[i]["ids"][0][0]

    true_source = metadata[i]["source"]
    pred_source = metadata[int(pred_id)]["source"]

    if str(true_id) == str(pred_id):
        chroma_insights["valid"] += 1

    elif true_source == pred_source:
        chroma_insights["similar"] += 1

    else:
        chroma_insights["invalid"] += 1

chroma_insights["valid_percentage"] = chroma_insights["valid"]/len(doc_questions)
chroma_insights["similar_percentage"] = chroma_insights["similar"]/len(doc_questions)
chroma_insights["invalid_percentage"] = chroma_insights["invalid"]/len(doc_questions)

print("Model ID:", model_id)
print("----")
print("Valid:", chroma_insights["valid"])
print("Valid%:", chroma_insights["valid_percentage"])
print("----")
print("Similar:", chroma_insights["similar"])
print("Similar%:", chroma_insights["similar_percentage"])
print("----")
print("InValid:", chroma_insights["invalid"])
print("InValid%:", chroma_insights["invalid_percentage"])
print("----")

Model ID: sentence-transformers/distiluse-base-multilingual-cased-v2
----
Valid: 748
Valid%: 0.18528610354223432
----
Similar: 514
Similar%: 0.12732226901164231
----
InValid: 2775
InValid%: 0.6873916274461234
----


In [ ]:
faiss_results = []

for i in range(len(doc_questions)):

    ques = encoded_questions[i].reshape(1, dim)

    faiss.normalize_L2(ques)

    scores, ids = faiss_index.search(ques, 3)

    faiss_results.append({
        "scores": scores,
        "ids": ids
    })


In [ ]:
faiss_insights = {
    "valid": 0,
    "similar": 0,
    "invalid": 0
}

for i in range(len(doc_questions)):
    true_id = docs_ids[i]
    pred_id = faiss_results[i]["ids"][0][0]

    true_source = metadata[i]["source"]
    pred_source = metadata[int(pred_id)]["source"]

    if str(true_id) == str(pred_id):
        faiss_insights["valid"] += 1

    elif true_source == pred_source:
        faiss_insights["similar"] += 1

    else:
        faiss_insights["invalid"] += 1


faiss_insights["valid_percentage"] = faiss_insights["valid"]/len(doc_questions)
faiss_insights["similar_percentage"] = faiss_insights["similar"]/len(doc_questions)
faiss_insights["invalid_percentage"] = faiss_insights["invalid"]/len(doc_questions)


print("Model ID:", model_id)
print("----")
print("Valid:", faiss_insights["valid"])
print("Valid%:", faiss_insights["valid_percentage"])
print("----")
print("Similar:", faiss_insights["similar"])
print("Similar%:", faiss_insights["similar_percentage"])
print("----")
print("InValid:", faiss_insights["invalid"])
print("InValid%:", faiss_insights["invalid_percentage"])
print("----")

Model ID: sentence-transformers/distiluse-base-multilingual-cased-v2
----
Valid: 907
Valid%: 0.2246717859796879
----
Similar: 638
Similar%: 0.15803814713896458
----
InValid: 2492
InValid%: 0.6172900668813476
----
